In [1]:
from typing import Callable, Any

import arff_parser as ap

In [2]:
census = ap.parse_arff_file("adult_train.arff")

census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,None,103497.0,Some-college,10.0,Never-married,None,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [3]:
def sum_attr_pred(func: Callable[[Any], bool], attr: str) -> int:
    return sum(func(x) for x in census[attr])

In [4]:
for attr in census.columns:
    print(f"# Of Instances with {attr} = '?': {sum_attr_pred(lambda x: x is None, attr)}")

# Of Instances with age = '?': 0
# Of Instances with workclass = '?': 2503
# Of Instances with fnlwgt = '?': 0
# Of Instances with education = '?': 0
# Of Instances with education-num = '?': 0
# Of Instances with marital-status = '?': 0
# Of Instances with occupation = '?': 2512
# Of Instances with relationship = '?': 0
# Of Instances with race = '?': 0
# Of Instances with sex = '?': 0
# Of Instances with capital-gain = '?': 0
# Of Instances with capital-loss = '?': 0
# Of Instances with hours-per-week = '?': 0
# Of Instances with native-country = '?': 766
# Of Instances with class = '?': 0


In [5]:
cnt = 0
total_rows = 0

for idx, row in census.iterrows():
    if any(x is None for x in row):
        cnt += 1
    total_rows += 1

print(f"{cnt} of total {total_rows} rows contain atleast one unknown (?) Attribute: {100 * (cnt / total_rows)}%")

3236 of total 43958 rows contain atleast one unknown (?) Attribute: 7.361572410027755%


In [6]:
def print_csv_line(file, data):
    file.write(", ".join(map(lambda x: '?' if x is None else str(x), data)))
    file.write('\n')

with open("processed.csv", "w+") as f:
    for idx, row in census.iterrows():
        print_csv_line(f, [
            row['age'],
            row['workclass'],
            row['education-num'],
            1 if row['marital-status'] in ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'] else 0,
            row['occupation'],
            row['relationship'],
            row['race'],
            row['sex'],
            row['capital-gain'],
            row['capital-loss'],
            row['hours-per-week'],
            1 if row['native-country'] == 'United-States' else 0,
            row['class']
        ])

In [7]:
usa_gt_50k = 0
usa_lt_50k = 0
for_gt_50k = 0
for_lt_50k = 0

for idx, row in census.iterrows():
    if row['native-country'] == 'United-States':
        if row['class'] == '>50K':
            usa_gt_50k += 1
        else:
            usa_lt_50k += 1
    else:
        if row['class'] == '>50K':
            for_gt_50k += 1
        else:
            for_lt_50k += 1

print(f"{100 * (usa_gt_50k / (usa_gt_50k + usa_lt_50k))}% of US. Citizens earn more than 50k / year")
print(f"{100 * (for_gt_50k / (for_gt_50k + for_lt_50k))}% of foreign Citizens earn more than 50k / year")

24.413597446679162% of US. Citizens earn more than 50k / year
19.754464285714285% of foreign Citizens earn more than 50k / year


In [15]:
rel_husband_gt_50k = 0
rel_husband_lt_50k = 0
rel_sum1 = 0
rel_wife_gt_50k = 0
rel_wife_lt_50k = 0
rel_sum2 = 0

ms_married_civ_gt_50k = 0
ms_married_civ_lt_50k = 0
ms_sum1 = 0
ms_married_af_gt_50k = 0
ms_married_af_lt_50k = 0
ms_sum2 = 0

num_husbands = 0
num_wifes = 0
num_men = 0
num_women = 0

num_unmarried_w_gt_50k = 0
num_unmarried_m_gt_50k = 0

for idx, row in census.iterrows():
    if row['relationship'] == 'Husband':
        if row['class'] == '>50K':
            rel_husband_gt_50k += 1
        else:
            rel_husband_lt_50k += 1
    elif row['relationship'] == 'Wife':
        if row['class'] == '>50K':
            rel_wife_gt_50k += 1
        else:
            rel_wife_lt_50k += 1
    if row['marital-status'] == 'Married-civ-spouse':
            if row['class'] == '>50K':
                ms_married_civ_gt_50k += 1
            else:
                ms_married_civ_lt_50k += 1
    elif row['marital-status'] == 'Married-AF-spouse':
                if row['class'] == '>50K':
                    ms_married_af_gt_50k += 1
                else:
                    ms_married_af_lt_50k += 1
    if row['sex'] == 'Male':
        num_men += 1
        if row['relationship'] == 'Husband':
            num_husbands += 1
        elif row['class'] == '>50K': 
            num_unmarried_m_gt_50k += 1
    else:
        num_women += 1
        if row['relationship'] == 'Wife':
            num_wifes += 1
        elif row['class'] == '>50K': 
            num_unmarried_w_gt_50k += 1
        
rel_sum1 = 100 * (rel_husband_gt_50k / (rel_husband_gt_50k + rel_husband_lt_50k))
print(f"{rel_sum1}% of husbands earn more than 50k / year")
rel_sum2 = 100 * (rel_wife_gt_50k / (rel_wife_gt_50k + rel_wife_lt_50k))
print(f"{rel_sum2}% of wifes earn more than 50k / year")
ms_sum1 = 100 * (ms_married_civ_gt_50k / (ms_married_civ_gt_50k + ms_married_civ_lt_50k))
print(f"{ms_sum1}% of married civilians earn more than 50k / year")
ms_sum2 = 100 * (ms_married_af_gt_50k / (ms_married_af_gt_50k + ms_married_af_lt_50k))
print(f"{ms_sum2}% of married armed forces earn more than 50k / year")

print(f"{(rel_sum1 + rel_sum2) / 2}% of the chosen relations earn more than 50k / year")
print(f"{(ms_sum1 + ms_sum2) / 2}% of the chosen marital-states earn more than 50k / year")

print(f"{num_men} Anz. Männer")
print(f"{num_husbands} Anz. Ehemänner")
print(f"{100 * (num_husbands / num_men)}% aller Männer sind verheiratet")
print(f"{num_unmarried_m_gt_50k} Anz. unverheirateter Männer > 50K Einkommen")
print(f"{num_women} Anz. Frauen")
print(f"{num_wifes} Anz. Ehefrauen")
print(f"{100 * (num_wifes/ num_women)}% aller Frauen sind verheiratet")
print(f"{num_unmarried_w_gt_50k} Anz. unverheirateter Frauen > 50K Einkommen ")


44.75119482710149% of husbands earn more than 50k / year
47.355769230769226% of wifes earn more than 50k / year
44.567374479063304% of married civilians earn more than 50k / year
40.625% of married armed forces earn more than 50k / year
46.05348202893536% of the chosen relations earn more than 50k / year
42.59618723953165% of the chosen marital-states earn more than 50k / year
29392 Anz. Männer
17784 Anz. Ehemänner
60.50626020685901 % verheirateter Männer
968 Anz. unverheirateter Männer > 50K Einkommen
14566 Anz. Frauen
2079 Anz. Ehefrauen
14.272964437731703 % verheirateter Frauen
612 Anz. unverheirateter Frauen > 50K Einkommen 


In [ ]:
rel_unmarried_gt_50k = 0
rel_unmarried_lt_50k = 0

ms_never_married_gt_50k = 0
ms_never_married_lt_50k = 0
ms_sum_1 = 0
ms_divorced_gt_50k = 0
ms_divorced_lt_50k = 0
ms_sum_2 = 0
ms_widowed_gt_50k = 0
ms_widowed_lt_50k = 0
ms_sum_3 = 0

for idx, row in census.iterrows():
    if row['relationship'] == 'Unmarried':
        if row['class'] == '>50K':
            rel_unmarried_gt_50k += 1
        else:
            rel_unmarried_lt_50k += 1
    elif row['material-status'] == 'Never-married':
        if row['class'] == '>50K':
            ms_never_married_gt_50k += 1
        else:
            ms_never_married_lt_50k += 1
    elif row['marital-status'] == 'Divorced':
            if row['class'] == '>50K':
                ms_divorced_gt_50k += 1
            else:
                ms_divorced_lt_50k += 1
    elif row['marital-status'] == 'Widowed':
                if row['class'] == '>50K':
                    ms_widowed_gt_50k += 1
                else:
                    ms_widowed_lt_50k += 1

print(f"{100 * (rel_unmarried_gt_50k / (rel_unmarried_gt_50k + rel_unmarried_lt_50k))}% of unmarried earn more than 50k / year")
ms_sum_1 = 100 * (ms_never_married_gt_50k / (ms_never_married_gt_50k + ms_never_married_lt_50k))
print(f"{ms_sum_1}% of never married earn more than 50k / year")
ms_sum_2 = 100 * (ms_divorced_gt_50k / (ms_divorced_gt_50k + ms_divorced_lt_50k))
print(f"{ms_sum_2}% of divorced earn more than 50k / year")
ms_sum_3 = 100 * (ms_widowed_gt_50k / (ms_widowed_gt_50k + ms_widowed_lt_50k))
print(f"{ms_sum_3}% of widowed earn more than 50k / year")

print(f"{(ms_sum_1 + ms_sum_2 + ms_sum_3) / 3}% of the unmarried marital-states earn more than 50k / year")
